In [48]:
import os
import glob
import numpy as np
import cv2

In [49]:
img_dir = '../images/'

In [50]:
ext_dict = {
    'og' : 'original/',
    'all' : 'all_edges/',
    'o1' : '0/',
    'o2' : '30/',
    'o3' : '60/',
    'o4' : '90/',
    'o5' : '120/',
    'o6' : '150/'
}

In [51]:
orientation_angle_dict = {
    'o1': 0, # 0 degrees
    'o2': np.pi / 6, # 30 degrees
    'o3': np.pi / 3, # 60 degrees
    'o4': np.pi / 2, # 90 degrees
    'o5': 2 * np.pi / 3, # 120 degrees
    'o6': 5 * np.pi / 6 # 150 degrees
}

In [52]:
for ext in ext_dict.values():
    if not os.path.exists(img_dir + ext):
        os.makedirs(img_dir + ext)

In [53]:
orientations = ['o1', 'o2', 'o3', 'o4', 'o5', 'o6']

In [54]:
def get_paths(dir, image_type='png'):
    """
    get the file name/path of all the files within a folder.
        e.g. glob.glob("/home/adam/*/*.txt").
    Use glob.escape to escape strings that are not meant to be patterns
        glob.glob(glob.escape(directory_name) + "/*.txt")
    :param directory_name: (str) the root directory name that contains all the images we want
    :param image: (str) either "jpg" or "png"
    :return: a list of queried files and directories
    """
    
    end = "/*." + image_type
    file_paths = glob.glob(glob.escape(dir) + end)

    # Sort the file paths numerically based on file names
    sorted_file_paths = sorted(file_paths, key=lambda x: int(x.split('/')[-1].split('.')[0]))
    
    return sorted_file_paths

In [55]:
originals = get_paths(img_dir + ext_dict['og'])

In [56]:
# intensity thresholds 
t1 = 10
t2 = 50
# angle error threshold
angle_error_threshold = np.pi / 12 # 15 degrees

In [57]:
def generate_edge_by_orientation(all_edges, gradient_angle, orientation):
    # Retrieve the orientation angle and the corresponding save ext
    orientation_angle = orientation_angle_dict[orientation]
    # subtract 90 degrees from the angle
    orientation_angle -= np.pi / 2
    # Compute orientation mask
    edge_mask = np.logical_and(gradient_angle > orientation_angle - angle_error_threshold,
                               gradient_angle < orientation_angle + angle_error_threshold)
    # Apply mask and save result
    result = all_edges.copy()
    result[~edge_mask] = 0
    cv2.imwrite(os.path.join(img_dir, ext_dict[o] + file_name), result)

In [58]:
# counter for saving imgs
i = 0
# For each original image...
for f in originals: 
    # Read image in greyscale
    img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    assert img is not None, "file could not be read, check with os.path.exists()"
    # Compute all edges and save result
    all_edges = cv2.Canny(image=img, threshold1=t1, threshold2=t2)
    file_name = str(i) + '.png'
    cv2.imwrite(os.path.join(
        img_dir, ext_dict['all'] + file_name), all_edges)
    # Compute gradient angle using Sobel with kernel size 3
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_angle = np.arctan2(sobely, sobelx)
    # For each orientation...
    for o in orientations:
        # Generate edges and save result
        generate_edge_by_orientation(all_edges, gradient_angle, o)
    i += 1